### 1. Shape Recognition

This code will recognize shapes using the Intel RealSense camera you can run this code using the run Button
When you run the code, 3 windows will appear :

A live color image from the camera - this is where you will see the final result

A black and white image from the camera

A window with 3 sliding bars.

In [3]:
import numpy as np
import cv2
import pyrealsense2 as rs

##### 1.1 Adjusting Values
Run the notebook using the run all button at the top. After you see the windows open, adjust the sliding bars until you can see the shapes clearly in the black and white image you should start seeing some rough outlines in the colored image

Note that the Low value HAS to be lower than the High value

For example - the Low H must always be lower than the High H in order to work

What values did you end at? Did you make any observations or notice any trends?
(Remove this line for your answer)

##### 1.2 Finish ReturnShape

Finish the ReturnShape function. It currently always returns circle. We want it to return the correct shape based on the number of sides. Ask instructors for help, and refer to slides for python syntax.

Hint : Use an IF, elif, else statement. First finish triangle, cuboid, and circle, and then try accounting for Triballs

In [4]:
def ReturnShape(NumberOfSides):
        return "circle"

In [5]:
#! STUDENTS DO NOT CHANGE ANYTHING FROM HERE ON
#! DO NOT TOUCH
def nothing(x):
    pass

# Create trackbar window with specified size
cv2.namedWindow('Trackbars', cv2.WINDOW_NORMAL)
# Set the desired width and height of the window
cv2.resizeWindow('Trackbars', 400, 300)

# Create trackbars for adjusting the HSV range
cv2.createTrackbar('High H', 'Trackbars', 179, 179, nothing)
cv2.createTrackbar('Low S', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('High S', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('Low V', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('High V', 'Trackbars', 255, 255, nothing)

# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Define font for text
font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    # Wait for a frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()

    if not color_frame:
        continue

    # Convert RealSense frame to OpenCV format
    frame = np.asanyarray(color_frame.get_data())

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get trackbar positions for HSV ranges
    low_h = cv2.getTrackbarPos('Low H', 'Trackbars')
    high_h = cv2.getTrackbarPos('High H', 'Trackbars')
    low_s = cv2.getTrackbarPos('Low S', 'Trackbars')
    high_s = cv2.getTrackbarPos('High S', 'Trackbars')
    low_v = cv2.getTrackbarPos('Low V', 'Trackbars')
    high_v = cv2.getTrackbarPos('High V', 'Trackbars')

    # Create lower and upper HSV boundaries
    lower_bound = np.array([low_h, low_s, low_v])
    upper_bound = np.array([high_h, high_s, high_v])

    # Threshold the HSV image to get only specified colors
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Find contours in the masked image
    contours, _ = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours, label shapes based on area and number of vertices
    for cnt in contours:
        area = cv2.contourArea(cnt)
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]

        if area > 400:
            cv2.drawContours(frame, [approx], 0, (0, 0, 0), 5)
            shape = ReturnShape(len(approx))
            cv2.putText(frame, shape, (x, y), font, 1, (0, 0, 0))

    cv2.imshow('Color Image', frame)
    cv2.imshow('mask', mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Stop streaming
pipeline.stop()
cv2.destroyAllWindows()

RuntimeError: Frame didn't arrive within 5000

: 